In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark'

In [2]:
# Importation

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext

# Instantiation
spark = SparkSession.builder.master("local").appName("FolksDF").getOrCreate()

creation de tables

In [3]:
l1 = [('75','Paris'),('51','Reims'),('06','Nice')]
rdd1 = spark.sparkContext.parallelize(l1)
Agence = rdd1.toDF(['Id','Adresse'])
Agence.createOrReplaceTempView('AgenceSQL')
Agence.show()

+---+-------+
| Id|Adresse|
+---+-------+
| 75|  Paris|
| 51|  Reims|
| 06|   Nice|
+---+-------+



In [4]:
l2 = [('1','10'),('2','20'),('3','60')]
rdd2 = spark.sparkContext.parallelize(l2) #On obtient un rdd
Prix = rdd2.toDF(['Idm','Prix'])
Prix.createOrReplaceTempView('PrixSQL')
Prix.show()

+---+----+
|Idm|Prix|
+---+----+
|  1|  10|
|  2|  20|
|  3|  60|
+---+----+



In [5]:
l3 = [('1','Sony','Souris'),('2','Samsung','Clavier'),('3','Mac','Ecran')]
rdd3 = spark.sparkContext.parallelize(l3) #On obtient un rdd
Produit = rdd3.toDF(['Idm','Marque','Produit'])
Produit.createOrReplaceTempView('ProduitSQL')
Produit.show()

+---+-------+-------+
|Idm| Marque|Produit|
+---+-------+-------+
|  1|   Sony| Souris|
|  2|Samsung|Clavier|
|  3|    Mac|  Ecran|
+---+-------+-------+



In [6]:
l4 = [('3','75','Toto','2','2021'),('2','75','Toto','1','2020'),('1','51','Toto','5','2021'),('2','51','Fofo','6','2021')]
rdd4 = spark.sparkContext.parallelize(l4) #On obtient un rdd\
Vente = rdd4.toDF(['Article','Agence','Client','Quantite','Annee'])
Vente.createOrReplaceTempView('VenteSQL')
Vente.show()

+-------+------+------+--------+-----+
|Article|Agence|Client|Quantite|Annee|
+-------+------+------+--------+-----+
|      3|    75|  Toto|       2| 2021|
|      2|    75|  Toto|       1| 2020|
|      1|    51|  Toto|       5| 2021|
|      2|    51|  Fofo|       6| 2021|
+-------+------+------+--------+-----+



Question 1: Déterminer les prix de chaque produit

In [7]:
# SQL
spark.sql(''' select Produit, Marque, Prix 
              from PrixSQL 
              join ProduitSQL 
              on PrixSQL.Idm = ProduitSQL.Idm ''').show()

+-------+-------+----+
|Produit| Marque|Prix|
+-------+-------+----+
|  Ecran|    Mac|  60|
| Souris|   Sony|  10|
|Clavier|Samsung|  20|
+-------+-------+----+



In [8]:
# DSL
Prix.join(Produit,['Idm']) \
          .select('Produit','Marque','Prix') \
          .show()

+-------+-------+----+
|Produit| Marque|Prix|
+-------+-------+----+
|  Ecran|    Mac|  60|
| Souris|   Sony|  10|
|Clavier|Samsung|  20|
+-------+-------+----+



 Question 2: Déterminer le prix d'une souris Sony

In [9]:
#SQL
spark.sql(''' select Produit, Marque, Prix 
               from PrixSQL 
               join ProduitSQL 
               on PrixSQL.Idm = ProduitSQL.Idm
               where Marque='Sony'
               and Produit='Souris' ''').show()

+-------+------+----+
|Produit|Marque|Prix|
+-------+------+----+
| Souris|  Sony|  10|
+-------+------+----+



In [10]:
# DSL
    #Solution 1\n",
Prix.join(Produit,['Idm']) \
        .select('Produit','Marque','Prix') \
        .filter(F.col('Produit')=='Souris') \
        .filter(F.col('Marque')=='Sony') \
        .show()
   


+-------+------+----+
|Produit|Marque|Prix|
+-------+------+----+
| Souris|  Sony|  10|
+-------+------+----+



In [11]:
   
#Solution 2 
Prix.join(Produit,['Idm']) \
       .select('Produit','Marque','Prix') \
       .filter((F.col('Produit')=='Souris') & (F.col('Marque')=='Sony')) \
        .show()
   

+-------+------+----+
|Produit|Marque|Prix|
+-------+------+----+
| Souris|  Sony|  10|
+-------+------+----+



"Question 3: Déterminer les articles que Toto a acheté en 2021"

In [12]:
#SQL
spark.sql(''' select Client, Produit
            from VenteSQL 
            join ProduitSQL 
            on VenteSQL.Article = ProduitSQL.Idm
            where Annee='2021'
            and Client='Toto' ''').show()
   

+------+-------+
|Client|Produit|
+------+-------+
|  Toto|  Ecran|
|  Toto| Souris|
+------+-------+



In [13]:
# DSL\n",
Vente.join(Produit,F.col('Article')==F.col('Idm')) \
    .select('Client','Produit') \
    .filter((F.col('Annee')=='2021') & (F.col('Client')=='Toto')) \
    .show()

+------+-------+
|Client|Produit|
+------+-------+
|  Toto|  Ecran|
|  Toto| Souris|
+------+-------+



"Question 4: Quel est le montant qu'a dépensé Toto en 2021?"

In [14]:
# SQL
#Solution 1\n",
spark.sql(''' select Client, sum(Quantite*Prix) as Depense
               from VenteSQL
               join PrixSQL 
               on VenteSQL.Article = PrixSQL.Idm
               where Annee='2021'
               and Client='Toto'
               group by Client ''').show()

+------+-------+
|Client|Depense|
+------+-------+
|  Toto|  170.0|
+------+-------+



In [15]:
# DSL\n",
Vente.join(Prix,F.col('Article')==F.col('Idm')) \
    .withColumn('Depense',F.col('Quantite')*F.col('Prix')) \
    .select('Client','Depense') \
    .filter((F.col('Annee')=='2021') & (F.col('Client')=='Toto')) \
    .groupBy('Client') \
    .agg(F.sum('Depense').alias('Depense')) \
     .show()

+------+-------+
|Client|Depense|
+------+-------+
|  Toto|  170.0|
+------+-------+



Question 5 : Quel est le montant depensé par Toto en 2021 par agence ?  (A FAIRE: SQL ET DSL)"

In [16]:
# DSL
Vente.join(Prix,F.col('Article')==F.col('Idm')) \
       .withColumn('Depense',F.col('Quantite')*F.col('Prix')) \
       .select('Client','Agence','Depense') \
       .filter((F.col('Annee')=='2021') & (F.col('Client')=='Toto')) \
       .groupBy('Client','Agence') \
       .agg(F.sum('Depense').alias('Depense')) \
       .show()

+------+------+-------+
|Client|Agence|Depense|
+------+------+-------+
|  Toto|    51|   50.0|
|  Toto|    75|  120.0|
+------+------+-------+



In [25]:
#SQL
spark.sql(''' select Client, Agence, sum(Quantite*prix) as Depense
              from VenteSQL
              join PrixSQL 
              on VenteSQL.Article = PrixSQL.Idm
              where Annee=='2021' and Client='Toto'
              group by Client, Agence
                ''').show()

+------+------+-------+
|Client|Agence|Depense|
+------+------+-------+
|  Toto|    51|   50.0|
|  Toto|    75|  120.0|
+------+------+-------+



Question 6 : Calculer le chiffre d'affaires par agence. (Quantite vendu * Prix)"

In [17]:
### SQL,
#Solution 1
spark.sql(''' select AgenceSQL.Adresse, req.Chiffre_Affaire
            from (select Agence, sum(Quantite*Prix) as Chiffre_Affaire
                from VenteSQL
                join PrixSQL 
                on VenteSQL.Article = PrixSQL.Idm
                group by Agence ) req
                join AgenceSQL on AgenceSQL.Id = req.Agence ''').show()

+-------+---------------+
|Adresse|Chiffre_Affaire|
+-------+---------------+
|  Reims|          170.0|
|  Paris|          140.0|
+-------+---------------+



In [18]:
#Solution 2
spark.sql(''' select A.Adresse, sum(Quantite*Prix) as Chiffre_Affaire
            from VenteSQL as V, PrixSQL as P, AgenceSQL as A
            where V.Article = P.Idm and A.Id = V.Agence
            group by Adresse ''').show()

+-------+---------------+
|Adresse|Chiffre_Affaire|
+-------+---------------+
|  Paris|          140.0|
|  Reims|          170.0|
+-------+---------------+



In [19]:
#Solution 3 (A TESTER)
spark.sql(''' select A.Adresse, sum(Quantite*Prix) as Chiffre_affaire
              from VenteSQL as v join PrixSQL as p on v.Article=p.Idm 
              join AgenceSQL as A on A.Id = v.Agence
              group by Adresse ''').show()

+-------+---------------+
|Adresse|Chiffre_affaire|
+-------+---------------+
|  Paris|          140.0|
|  Reims|          170.0|
+-------+---------------+



In [21]:
# DSL
Vente.join(Prix,F.col('Idm')==F.col('Article')) \
    .join(Agence,F.col('Id')==F.col('Agence')) \
    .withColumn('CA',F.col('Quantite')*F.col('Prix')) \
    .groupBy('Adresse') \
    .agg(F.sum('CA').alias('Chiffre_Affaire')) \
    .show()

+-------+---------------+
|Adresse|Chiffre_Affaire|
+-------+---------------+
|  Paris|          140.0|
|  Reims|          170.0|
+-------+---------------+



Question 7 : Quel est le produit le plus acheté en 2021? SQL + DSL

In [17]:
#sql
spark.sql(''' select Produit, sum(Quantite) as Q1 
               from VenteSQL
               join ProduitSQL 
               on VenteSQL.Article = ProduitSQL.Idm
               where Annee='2021'
               group by Produit
               order by Q1 desc''').first()


Row(Produit='Clavier', Q1=6.0)

In [54]:
#dsl
# DSL
Vente.join(Produit,F.col('Article')==F.col('Idm')) \
     .select('Produit','Annee', 'Quantite') \
     .filter((F.col('Annee')=='2021')) \
     .orderBy (F.col('Quantite').desc())\
     .first()

Row(Produit='Clavier', Annee='2021', Quantite='6')

Question 8 : Quel est le client qui a acheté le plus d'articles? SQL + DSL

In [72]:
#SQL

spark.sql(''' select Client, sum(Quantite) as QuanTotal
              from VenteSQL
              group by Client
              order by QuanTotal desc
              ''').first()

Row(Client='Toto', QuanTotal=8.0)

In [79]:
#DSl
Vente.select('Client','Quantite') \
    .groupBy('Client') \
    .agg(F.sum('Quantite').alias('QTotal'))\
    .orderBy (F.col('QTotal').desc())\
    .first()

Row(Client='Toto', QTotal=8.0)

Question 9 : Quel est le client qui a dépensé le plus d'argent? SQL + DSL

In [13]:
#SQL
spark.sql(''' select Client, sum(Quantite*Prix) as Depense
               from VenteSQL
               join PrixSQL 
               on VenteSQL.Article = PrixSQL.Idm
               group by Client 
               order by Depense desc''').first()

Row(Client='Toto', Depense=190.0)

In [12]:
#DSL
Vente.join(Prix,F.col('Article')==F.col('Idm')) \
    .withColumn('Depense',F.col('Quantite')*F.col('Prix')) \
    .select('Client','Depense') \
    .groupBy('Client') \
    .agg(F.sum('Depense').alias('Depense')) \
    .orderBy (F.col('Depense').desc())\
    .first()

Row(Client='Toto', Depense=190.0)